In [ ]:
#http://stackoverflow.com/questions/32503826/how-can-i-grab-data-series-from-xml-or-tcx-file
#https://github.com/vkurup/python-tcxparser/blob/master/tcxparser.py
#http://www.diveintopython3.net/xml.html
#TODO add wind bearing relative to rider
#TODO create class based structure for the data and functions!
#imports
import numpy as np
import tcxparser as TCX
import csv,json,requests
from pprint import pprint

In [ ]:
#Read the data
xmlfile = 'demoRoute.tcx'
data = TCX.TCXParser(xmlfile)
end = len(data.time_values())
samplePoints = 10
ind = np.linspace(0, (end-1),samplePoints, endpoint=True, retstep=False, dtype=None)
ind = np.floor(ind)
ind = ind.astype(int)
latitude =data.latitude_points()
longitude = data.longitude_points()
distance = data.distance_points()
lat = np.array(latitude)
lat = lat[np.ix_ (ind)]
lon = np.array(longitude)
lon = lon[np.ix_ (ind)]

In [ ]:
lat = lat[4]
lon = lon[4]
units = 'si'

url = 'https://api.darksky.net/forecast/***REMOVED***/{0},{1}?units={2}'.format(lat,lon,units)

retrieved = requests.get(url)
file = open('weatherdataTest.json','wb') 
file.write(retrieved.content) 
file.close()

In [ ]:
with open('weatherdataTest.json') as data_file:    
    weatherdata = json.load(data_file)


In [ ]:
hourlyData=weatherdata['hourly']
pprint(hourlyData)

In [ ]:
#pip install gpxpy or the maths!
#https://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html
#not actually needed as Strava has already supplied this data
def distance(lat1,lat2,lon1,lon2):
    "This calculates distance between two coordinates"
    R = 6371.008 #mean earth radius in km
    (latR1,latR2,lonR1,lonR2) = [np.deg2rad(x) for x in (lat1,lat2,lon1,lon2)]
    dlat = latR2-latR1
    dlon = lonR2-lonR1
    a = np.sin(dlat / 2)**2 + np.cos(latR1) * np.cos(latR2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    dist = R*c
    return dist;
